In [62]:
import sys
from pathlib import Path
import pandas as pd 
import numpy as np
import keras
import math 
import tensorflow as tf
from keras.layers import LSTM, Dense, Dropout, LeakyReLU, GRU, BatchNormalization, Input, LayerNormalization
from keras.regularizers import l1, l2, l1_l2
from tensorflow.keras.optimizers import SGD
import tensorflow_addons as tfa
from keras.models import Sequential
from keras.callbacks import LearningRateScheduler, EarlyStopping
from keras.preprocessing.sequence import TimeseriesGenerator
sys.path.append(str(Path("../..").resolve()))
from src.constants import model_data_dir, data_dir, raw_data_dir, raw_data_name
from src.utils import use_target

In [15]:
df_train = pd.read_csv(model_data_dir / "train_classification.csv")
df_test = pd.read_csv(model_data_dir / "test_classification.csv")
test_dates = pd.to_datetime(df_test["date"])
df_train = df_train.drop(columns=["date"])
df_test = df_test.drop("date", axis=1)
df_train = df_train.loc[:, ~df_train.columns.str.contains("_mv_")]
df_test = df_test.loc[:, ~df_test.columns.str.contains("_mv_")]
df_train = pd.get_dummies(df_train.astype({
    "month": "category",
    "weekday": "category",
    "day": "category",
}), drop_first=True)
df_test = pd.get_dummies(df_test.astype({
    "month": "category",
    "weekday": "category",
    "day": "category",
}), drop_first=True)

## LSTM

In [9]:
win_length = 40
batch_size = 10
num_features = df_train.shape[1] - 3

In [16]:
model = Sequential()
model.add(Input(shape=(win_length, num_features)))
# model.add(LSTM(64, return_sequences=True, bias_initializer="zeros", unit_forget_bias=True, kernel_regularizer=l1(1e-4), recurrent_regularizer=l2(2e-4)))
# model.add(LeakyReLU(alpha=0.5)) 
# model.add(LayerNormalization())
# model.add(Dropout(0.3)) 
model.add(LSTM(20, return_sequences=False, bias_initializer="zeros", unit_forget_bias=True, kernel_regularizer=l1(1e-4), recurrent_regularizer=l2(2e-4)))
model.add(LeakyReLU(alpha=0.5)) 
model.add(LayerNormalization())
model.add(Dropout(0.3)) 
model.add(Dense(1, activation="sigmoid", kernel_regularizer=l1_l2(1e-4, 2e-4)))

In [17]:
early_stop = EarlyStopping(monitor = "loss",
                           patience = 5)

step = tf.Variable(0, trainable=False)
schedule = tf.optimizers.schedules.PiecewiseConstantDecay(
    [10000, 15000], [1e-0, 1e-1, 1e-2])
# lr and wd can be a function or a tensor
lr = 1e-5 * schedule(step)
wd = lambda: 1e-4 * schedule(step)
model.compile(loss=keras.losses.binary_crossentropy,
            #   optimizer=tfa.optimizers.SGDW(
            #   learning_rate=lr, 
            #   weight_decay=wd, 
            #   momentum=0.9),
              optimizer=tfa.optimizers.AdamW(learning_rate=lr, weight_decay=wd), 
              metrics=["accuracy"])

## size

In [18]:
y_train_size, x_train_size = use_target(df_train, "sc_1d_fwd_rel_d", "classification")
y_test_size, x_test_size = use_target(df_test, "sc_1d_fwd_rel_d", "classification")

train_generator_size = TimeseriesGenerator(x_train_size, y_train_size, length=win_length, sampling_rate=1, batch_size=batch_size)
test_generator_size = TimeseriesGenerator(x_test_size, y_test_size, length=win_length, sampling_rate=1, batch_size=batch_size)

In [19]:
history = model.fit(train_generator_size, 
                    epochs=20,
                    validation_data=test_generator_size,
                    shuffle=False,
                    callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)])

Epoch 1/20
334/334 [==============================] - 3s 6ms/step - loss: 1.1353 - accuracy: 0.5112 - val_loss: 0.8746 - val_accuracy: 0.5038
Epoch 2/20
334/334 [==============================] - 2s 6ms/step - loss: 0.9659 - accuracy: 0.5112 - val_loss: 0.8806 - val_accuracy: 0.5049
Epoch 3/20
334/334 [==============================] - 2s 6ms/step - loss: 0.9043 - accuracy: 0.5214 - val_loss: 0.8722 - val_accuracy: 0.5043
Epoch 4/20
334/334 [==============================] - 2s 6ms/step - loss: 0.8829 - accuracy: 0.5121 - val_loss: 0.8478 - val_accuracy: 0.4989
Epoch 5/20
334/334 [==============================] - 2s 6ms/step - loss: 0.8545 - accuracy: 0.5034 - val_loss: 0.8322 - val_accuracy: 0.4978
Epoch 6/20
334/334 [==============================] - 2s 6ms/step - loss: 0.8351 - accuracy: 0.5109 - val_loss: 0.8218 - val_accuracy: 0.5011
Epoch 7/20
334/334 [==============================] - 2s 6ms/step - loss: 0.8250 - accuracy: 0.5034 - val_loss: 0.8131 - val_accuracy: 0.5054
Epoch 

In [21]:
size_probs = model.predict(test_generator_size).flatten()
size_preds = np.where(size_probs > 0.5, 1, 0)
true = y_test_size[0:len(y_test_size) - win_length ]
np.mean(np.where(size_preds > 0.5, 1, 0) == true)

0.5005417118093174

## momentum

In [22]:
y_train_mom, x_train_mom = use_target(df_train, "mom_1d_fwd_rel_d", "classification")
y_test_mom, x_test_mom = use_target(df_test, "mom_1d_fwd_rel_d", "classification")

train_generator_mom = TimeseriesGenerator(x_train_mom, y_train_mom, length=win_length, sampling_rate=1, batch_size=batch_size)
test_generator_mom = TimeseriesGenerator(x_test_mom, y_test_mom, length=win_length, sampling_rate=1, batch_size=batch_size)

In [23]:
model = Sequential()
model.add(Input(shape=(win_length, num_features)))
model.add(LSTM(20, bias_initializer="zeros", unit_forget_bias=True, kernel_regularizer=l1(1e-4), recurrent_regularizer=l2(2e-4)))
model.add(LeakyReLU(alpha=0.5)) 
model.add(LayerNormalization())
model.add(Dropout(0.3)) 
model.add(Dense(1, activation="sigmoid", kernel_regularizer=l1_l2(1e-4, 2e-4)))

In [24]:
early_stop = EarlyStopping(monitor = "loss",
                           patience = 5)

step = tf.Variable(0, trainable=False)
schedule = tf.optimizers.schedules.PiecewiseConstantDecay(
    [10000, 15000], [1e-0, 1e-1, 1e-2])
# lr and wd can be a function or a tensor
lr = 1e-5 * schedule(step)
wd = lambda: 1e-4 * schedule(step)
model.compile(loss=keras.losses.binary_crossentropy,
            #   optimizer=tfa.optimizers.SGDW(
            #   learning_rate=lr, 
            #   weight_decay=wd, 
            #   momentum=0.9),
              optimizer=tfa.optimizers.AdamW(learning_rate=lr, weight_decay=wd), 
              metrics=["accuracy"])

In [25]:
history = model.fit(train_generator_mom, 
                    epochs=20,
                    validation_data=test_generator_mom,
                    shuffle=False,
                    callbacks=[early_stop])

Epoch 1/20
334/334 [==============================] - 4s 8ms/step - loss: 1.1045 - accuracy: 0.5112 - val_loss: 0.8621 - val_accuracy: 0.5336
Epoch 2/20
334/334 [==============================] - 2s 7ms/step - loss: 0.9603 - accuracy: 0.5064 - val_loss: 0.8436 - val_accuracy: 0.5379
Epoch 3/20
334/334 [==============================] - 2s 7ms/step - loss: 0.8999 - accuracy: 0.5001 - val_loss: 0.8330 - val_accuracy: 0.5379
Epoch 4/20
334/334 [==============================] - 2s 7ms/step - loss: 0.8802 - accuracy: 0.4933 - val_loss: 0.8233 - val_accuracy: 0.5406
Epoch 5/20
334/334 [==============================] - 2s 7ms/step - loss: 0.8500 - accuracy: 0.4990 - val_loss: 0.8163 - val_accuracy: 0.5455
Epoch 6/20
334/334 [==============================] - 2s 7ms/step - loss: 0.8372 - accuracy: 0.5100 - val_loss: 0.8094 - val_accuracy: 0.5433
Epoch 7/20
334/334 [==============================] - 2s 7ms/step - loss: 0.8219 - accuracy: 0.5118 - val_loss: 0.8024 - val_accuracy: 0.5439
Epoch 

In [47]:
mom_probs = model.predict(test_generator_mom).flatten()
mom_preds = np.where(mom_probs > 0.5, 1, 0)
true = y_test_mom[0:len(y_test_mom) - win_length ]
np.mean(mom_preds == true)

0.5433369447453954

## value

In [26]:
y_train_value, x_train_value = use_target(df_train, "value_1d_fwd_rel_d", "classification")
y_test_value, x_test_value = use_target(df_test, "value_1d_fwd_rel_d", "classification")

train_generator_value = TimeseriesGenerator(x_train_value, y_train_value, length=win_length, sampling_rate=1, batch_size=batch_size)
test_generator_value = TimeseriesGenerator(x_test_value, y_test_value, length=win_length, sampling_rate=1, batch_size=batch_size)

In [27]:
model = Sequential()
model.add(Input(shape=(win_length, num_features)))
model.add(LSTM(20, return_sequences=False, bias_initializer="zeros", unit_forget_bias=True, kernel_regularizer=l1(1e-4), recurrent_regularizer=l2(2e-4)))
model.add(LeakyReLU(alpha=0.5)) 
model.add(LayerNormalization())
model.add(Dropout(0.3)) 
model.add(Dense(1, activation="sigmoid", kernel_regularizer=l1_l2(1e-4, 2e-4)))

In [29]:
early_stop = EarlyStopping(monitor = "loss",
                           patience = 5)

step = tf.Variable(0, trainable=False)
schedule = tf.optimizers.schedules.PiecewiseConstantDecay(
    [10000, 15000], [1e-0, 1e-1, 1e-2])
# lr and wd can be a function or a tensor
lr = 1e-5 * schedule(step)
wd = lambda: 1e-4 * schedule(step)
model.compile(loss=keras.losses.binary_crossentropy,
            #   optimizer=tfa.optimizers.SGDW(
            #   learning_rate=lr, 
            #   weight_decay=wd, 
            #   momentum=0.9),
              optimizer=tfa.optimizers.AdamW(learning_rate=lr, weight_decay=wd), 
              metrics=["accuracy"])

In [30]:
history = model.fit(train_generator_mom, 
                    epochs=20,
                    validation_data=test_generator_mom,
                    shuffle=False,
                    callbacks=[early_stop])

Epoch 1/20
334/334 [==============================] - 4s 8ms/step - loss: 1.1717 - accuracy: 0.4819 - val_loss: 0.9044 - val_accuracy: 0.5054
Epoch 2/20
334/334 [==============================] - 2s 7ms/step - loss: 1.0000 - accuracy: 0.4978 - val_loss: 0.8727 - val_accuracy: 0.5195
Epoch 3/20
334/334 [==============================] - 2s 7ms/step - loss: 0.9329 - accuracy: 0.4906 - val_loss: 0.8507 - val_accuracy: 0.5271
Epoch 4/20
334/334 [==============================] - 2s 7ms/step - loss: 0.8841 - accuracy: 0.5001 - val_loss: 0.8347 - val_accuracy: 0.5320
Epoch 5/20
334/334 [==============================] - 2s 6ms/step - loss: 0.8590 - accuracy: 0.5013 - val_loss: 0.8210 - val_accuracy: 0.5325
Epoch 6/20
334/334 [==============================] - 2s 6ms/step - loss: 0.8382 - accuracy: 0.5043 - val_loss: 0.8099 - val_accuracy: 0.5341
Epoch 7/20
334/334 [==============================] - 2s 6ms/step - loss: 0.8381 - accuracy: 0.4999 - val_loss: 0.8013 - val_accuracy: 0.5330
Epoch 

In [50]:
value_probs = model.predict(test_generator_value).flatten()
value_preds = np.where(value_probs > 0.6, 1, 0)
true = y_test_value[0:len(y_test_value) - win_length ]
np.mean(value_preds == true)

0.5113759479956663

In [79]:
sp = pd.read_excel(raw_data_dir / raw_data_name, sheet_name=1, usecols=["Date", "S&P 500"])
sp["Date"] = pd.to_datetime(sp["Date"])
sp = sp.query("Date > '2013-12-31'").drop("Date", axis=1).pct_change().rename({"S&P 500": "s_&_p_500_ret"}, axis=1).reset_index(drop=True)


ret = pd.read_excel(raw_data_dir / raw_data_name, sheet_name=3, usecols=["Date", "sc_1d_fwd_ret", "mom_1d_fwd_ret", "value_1d_fwd_ret"])
ret["Date"] = pd.to_datetime(ret["Date"])
ret = ret.query("Date > '2013-12-31'").drop("Date",axis=1).reset_index(drop=True)

In [84]:
rows = df_test.shape[0]


preds = pd.DataFrame({
    "sc": df_test["sc_1d_fwd_rel_d"][:rows-win_length], 
    "mom": df_test["mom_1d_fwd_rel_d"][:rows-win_length],
    "value": df_test["value_1d_fwd_rel_d"][:rows-win_length],
    ".pred_prob_sc_lstm": size_probs,
    ".pred_prob_mom_lstm": mom_probs,
    ".pred_prob_value_lstm": value_probs, 
    ".pred_sc_lstm": size_preds,
    ".pred_mom_lstm": mom_preds,
    ".pred_value_lstm": value_preds,
    "s_&_p_500_ret": sp.iloc[:sp.shape[0]-win_length-1]["s_&_p_500_ret"]
})
lstm_preds = pd.concat([preds, ret.iloc[:ret.shape[0]-win_length]], axis=1)



lstm_preds
lstm_preds.to_csv(data_dir / "pred" / "lstm_preds.csv", index=False)

## GRU 

In [55]:
model_gru = Sequential()
model_gru.add(Input(shape=(win_length, num_features)))
model_gru.add(GRU(20, return_sequences=True))
model_gru.add(Dropout(0.3))
model_gru.add(GRU(units=30, return_sequences=True))
model_gru.add(GRU(units=30, return_sequences=False))
model_gru.add(Dropout(0.3))
model_gru.add(Dense(units=1, activation="sigmoid"))

In [58]:
early_stop = EarlyStopping(monitor = "loss",
                           patience = 5)

step = tf.Variable(0, trainable=False)
schedule = tf.optimizers.schedules.PiecewiseConstantDecay(
    [10000, 15000], [1e-0, 1e-1, 1e-2])
# lr and wd can be a function or a tensor
lr = 1e-5 * schedule(step)
wd = lambda: 1e-4 * schedule(step)
model_gru.compile(loss=keras.losses.binary_crossentropy,
              optimizer=tfa.optimizers.AdamW(learning_rate=lr, weight_decay=wd), 
              metrics=["accuracy"])

In [59]:
history = model_gru.fit(train_generator_mom, 
                    epochs=20,
                    validation_data=test_generator_mom,
                    shuffle=False,
                    callbacks=[early_stop])

Epoch 1/20
334/334 [==============================] - 9s 19ms/step - loss: 0.7095 - accuracy: 0.4837 - val_loss: 0.7005 - val_accuracy: 0.4599
Epoch 2/20
334/334 [==============================] - 6s 17ms/step - loss: 0.6969 - accuracy: 0.5046 - val_loss: 0.6924 - val_accuracy: 0.5206
Epoch 3/20
334/334 [==============================] - 6s 17ms/step - loss: 0.6965 - accuracy: 0.5085 - val_loss: 0.6903 - val_accuracy: 0.5493
Epoch 4/20
334/334 [==============================] - 6s 18ms/step - loss: 0.6946 - accuracy: 0.5130 - val_loss: 0.6895 - val_accuracy: 0.5444
Epoch 5/20
334/334 [==============================] - 6s 17ms/step - loss: 0.6898 - accuracy: 0.5355 - val_loss: 0.6893 - val_accuracy: 0.5433
Epoch 6/20
334/334 [==============================] - 6s 17ms/step - loss: 0.6918 - accuracy: 0.5301 - val_loss: 0.6893 - val_accuracy: 0.5439
Epoch 7/20
334/334 [==============================] - 6s 17ms/step - loss: 0.6910 - accuracy: 0.5280 - val_loss: 0.6892 - val_accuracy: 0.5428

In [60]:
mom_preds2 = np.where(model_gru.predict(test_generator_mom).flatten(), 1, 0)
true = y_test_mom[0:len(y_test_mom) - win_length]
np.mean(mom_preds2 == true)

0.5449620801733478